In [ ]:
import numpy as np

In [ ]:
###----------EJERCICIO 1-------------------

def cargarDataset():
    base = "template-alumnos/dataset/cats_and_dogs"

    # --- TRAIN ---
    X_cats = np.load(f"{base}/train/cats/efficientnet_b3_embeddings.npy")   
    X_dogs = np.load(f"{base}/train/dogs/efficientnet_b3_embeddings.npy")   

    X_train = np.hstack([X_cats, X_dogs])  

    Y_train = np.hstack([
        np.tile([[1],[0]], (1, X_cats.shape[1])),  # gatos
        np.tile([[0],[1]], (1, X_dogs.shape[1]))   # perros
    ])                                             

    # --- VAL ---
    V_cats = np.load(f"{base}/val/cats/efficientnet_b3_embeddings.npy")
    V_dogs = np.load(f"{base}/val/dogs/efficientnet_b3_embeddings.npy")

    X_val = np.hstack([V_cats, V_dogs])
    Y_val = np.hstack([
        np.tile([[1],[0]], (1, V_cats.shape[1])),
        np.tile([[0],[1]], (1, V_dogs.shape[1]))
    ])

    print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
    print("X_val:", X_val.shape, "Y_val:", Y_val.shape)

    return X_train, Y_train, X_val, Y_val

In [ ]:
X_t, Yt, Xv, Yv = cargarDataset()

Xt_trans = np.array(transpuesta(X_t))
Yt_trans  = transpuesta(Yt)
Xv_trans  = transpuesta(Xv)
Yv_trans  = transpuesta(Yv)

xtx = Xt_trans@ X_t ## matmul(Xt_trans,Xt)

X_train: (1536, 2000) Y_train: (2, 2000)
X_val: (1536, 1000) Y_val: (2, 1000)


In [ ]:



##   Z = foward_sub(L,X)
##   U = back_sub(transpose(L), Z)
#    W = Y @ tranpose(U)




def pinvEcuacionesNormales(L, Y):
    
    X = X_t
    
    n = len(L)       
    p = len(L[0])     

    # CASO (a): n > p  ---> usamos pseudo inversa = (X^T X)^(-1)X^T (1)
    if n > p:
        # Acá resolvemos que  (X^T X)U = X^T (2)

        """"

        Wx = Y --> W = Yx^+

        Sacando de (1) U = (X^T X)^(-1)X^T entonce resolvemos el sistema de (2)

        """

        Xt = transpuesta(X)           

        #  Resolvemos L Z = X^T
        Z = sust_adelante(L, Xt)      

        #  Resolvemos L^T U = Z
        Lt = transpuesta(L)
        U = sust_atras(Lt, Z)     #

        # Ahora U = (X^T X)^{-1} X^T  (p x n)

        # W = Y U
        W = matmul(Y, U)

    # CASO (b): n < p  ---> usamos pseudo inversa = X^T (XX^T)^(-1) (3)

    elif n < p:


        # Acá asumimos que V(XX^T) = X^T (4)

        """"

        Wx = Y --> W = Yx^+

        Sacando de (4) V(XX^T) = X^T ---> quiero q me quede algo asi (XX^T)V = X^T --> transpongo a ambos lados --> (XX^T)V^T = X

        """

        #  Resolvemos L Z = X
        Z = sust_adelante(L, X)       

        # Resolvemos L^T V = Z
        Lt = transpuesta(L)
        V_t = sust_atras(Lt, Z)     

        # Ahora V = (X X^T)^{-1} X   

        #  W = Y V^T

        V = transpuesta(V)           
        W = matmul(Y, V_t)

    else:

        X_inv = inversa(X) ## Falta hacer
        W = matmul(Y, X_inv) 

    return W

In [ ]:
## AGREGAR AL MODULO ALC

## MULTIPLICACION DE MATRICES

def matmul(A, B):

    m = len(A)
    n = len(A[0])

    
    if len(B) != n:
        return "Dimensiones incompatibles"
    
    p = len(B[0])

    # inicializo matriz C de ceros m x p
    C = []
    for _ in range(m):
        C.append([0.0] * p)

    # calculo C
    for i in range(m):
        for k in range(n):
            aik = A[i][k]
            for j in range(p):
                C[i][j] += aik * B[k][j]

    return C

## SUSTITUCION HACIA ADELANTE
def sust_adelante(L,X):
    z = []

    z.append(X[0] / L[0][0])

    n = L.shape[0]

    for i in range(n):
        suma_cuadrados = 0
        for j in range(i+1):

            suma_cuadrados += (L[i][j]*z[j])
            
        termino = X[i] - suma_cuadrados
        z.append(termino/L[i][i])

    return z

## SUSTITUCION HACIA ATRAS
def sust_atras(L, Z):
    U = []

    n = L.shape[0]

    for i in range(n-1, -1, -1):
        suma_cuadrados = 0
        for j in range(i+1, n):

            suma_cuadrados += (L[i][j]*U[j])
            
        termino = Z[i] - suma_cuadrados
        U.append(termino/L[i][i])

    U = U[::-1] ## INVEIRTO U PORQUE ME QUEDO TODO AL REVES

    return U

def cholesky(A):
    
    L = np.zeros_like(A)
    n = L.shape[0]

    for i in range(1,n):
    
    # Bucle 2 (columnas, hasta la diagonal)
        for j in range(i + 1):
        
        # --- CASO DIAGONAL (i == j) ---
            if i == j:
                suma_cuadrados = 0
            
            # Bucle 3 (para la sumatoria)
                for k in range(j):  
                    suma_cuadrados += L[i][k]**2
                
                termino = A[i][i] - suma_cuadrados
            
                if termino < 0:
                    pass
            
            L[i][i] =np.sqrt(termino)

        # --- CASO VERTICAL (i > j) ---
        else:
            producto_punto = 0
            
            # Bucle 3 (para la sumatoria)
            for k in range(j):  # k va de 0 a j-1
                producto_punto += L[i][k] * L[j][k]
            
            
            L[i][j] = (A[i][j] - producto_punto) / L[j][j]

    return L


## MULTIPLICACION DE MATRICES
def matmul(A, B):

    m = len(A)
    n = len(A[0])

    
    if len(B) != n:
        return "Dimensiones incompatibles"
    
    p = len(B[0])

    # inicializo matriz C de ceros m x p
    C = []
    for _ in range(m):
        C.append([0.0] * p)

    # calculo C
    for i in range(m):
        for k in range(n):
            aik = A[i][k]
            for j in range(p):
                C[i][j] += aik * B[k][j]

    return C


def transpuesta(M):
    filas = len(M)
    cols = len(M[0])
    T = [[0]*filas for _ in range(cols)]
    for i in range(filas):
        for j in range(cols):
            T[j][i] = M[i][j]
    return T

